In [ ]:
Create 만들고 Furniture 만들고 List 만들고 Modify 만들고 원래코드랑 봉합

In [26]:
import numpy as np
import pandas as pd
from tkinter import *
from tkinter import colorchooser

In [185]:
class Utils():
    def __init__(self):
        pass
    
    @staticmethod
    def size_to_geometry(window_width=100, window_height=100, window_x=0, window_y=0):
        size_geo = str(window_width) + 'x' + str(window_height)
        position_geo = str(window_x) + '+' + str(window_y)
        geometry = size_geo + '+' + position_geo
        return geometry
    
    @staticmethod
    def create_window(title, geometry, is_resize=False):
        window = Tk()
        window.geometry(geometry)
        window.title(title)
        window.resizable(width=is_resize, height=is_resize)
        return window
    
    @staticmethod
    def create_canvas(window, width=1, height=1, highlight=0, bg='default', x=0, y=0):
        if bg == 'default':
            bg = window.cget('bg')
            
        canvas = Canvas(window, width=width, height=height, highlightthickness=highlight, bg=bg)
        canvas.place(x=x, y=y)
        return canvas
    
    @staticmethod
    def create_label(window, default_text='', font='Courier', font_size=14, bg='default', x=0, y=0):
        if bg=='default':
            bg = window.cget('bg')
            
        label = Label(window, text=default_text, bg=bg, font=(font, font_size))
        label.place(x=x, y=y)
        return label
    
    @staticmethod
    def create_textbox(window, width=1, height=1, default_text='', font='Courier', font_size=14, x=0, y=0):
        textbox = Text(window, height=height, width=width, font=(font, font_size))
        textbox.place(x=x, y=y)
        textbox.insert(1.0, default_text)
        return textbox
    
    @staticmethod
    def get_textbox(textbox, to_int=False):
        txt = textbox.get('1.0', 'end').replace('\n', '')
        
        if to_int: return int(txt)
        else: return txt
        
    @staticmethod
    def set_textbox(textbox, txt=''):
        textbox.delete('1.0', 'end')
        textbox.insert(1.0, txt)

In [112]:
class GridModule():
    def __init__(self, window, parent, grid_x=0, grid_y=0, grid_width=100, grid_height=100, 
                cnt_grid_x=10, cnt_grid_y=10, max_grid=30, min_grid=2):
        self.window = window
        self.parent = parent
        
        self.frame_position = {'x':grid_x, 'y':grid_y}
        self.frame_size = {'w':grid_width, 'h':grid_height}
        self.grid_count = {'x':cnt_grid_x, 'y':cnt_grid_y}
        self.grid_size = self.get_gridsize()
        self.grid_limit = {'max':max_grid, 'min':min_grid}
        self.padding = {'canvas':20, 'grid':5}
        
        self.canvas = None
        self.make_grid()
        
    def get_gridsize(self):
        grid_width = self.frame_size['w'] / self.grid_count['x']
        grid_height = self.frame_size['h'] / self.grid_count['y']
        return {'w':grid_width, 'h':grid_height}
    
    def get_config(self):
        data = [
            {'position':self.frame_position},
            {'size':self.frame_size},
            {'grid_count':self.grid_count},
            {'grid_size':self.grid_size}
        ]
        
        return data
    
    def get_canvas_size(self):
        minx = self.frame_position['x']
        miny = self.frame_position['y']
        maxx = self.frame_position['x'] + self.frame_size['w']
        maxy = self.frame_position['y'] + self.frame_size['h']
        return {'min':{'x':minx,'y':miny}, 'max':{'x':maxx,'y':maxy}}
    
    def update_gridsize(self):
        self.grid_size = self.get_gridsize()
    
    def make_grid(self, grid_color='white'):
        if self.canvas != None:
            self.canvas.destroy()
            
        self.canvas = Utils.create_canvas(window=self.window, 
                                          width=self.frame_size['w'] + self.padding['canvas'], 
                                          height=self.frame_size['h'] + self.padding['canvas'], 
                                          x=self.frame_position['x'], y=self.frame_position['y'])
        for i in range(self.grid_count['x']):
            for j in range(self.grid_count['y']):
                self.canvas.create_rectangle(i*self.grid_size['w']+self.padding['grid'], j*self.grid_size['h']+self.padding['grid'],
                                            (i+1)*self.grid_size['w']+self.padding['grid'],
                                            (j+1)*self.grid_size['h']+self.padding['grid'], fill=grid_color)

In [171]:
class CreateModule():
    def __init__(self, window, parent, create_x=0, create_y=0, create_width=100, create_height=100, bg='cyan'):
        self.window = window
        self.parent = parent
        self.canvas = Utils.create_canvas(window=window, width=create_width, height=create_height,
                                         bg=bg, x=create_x, y=create_y)
        
        Utils.create_label(window=self.canvas, default_text='Size', font='Courier', font_size=20, x=30, y=150)
        Utils.create_label(window=self.canvas, default_text='X', font='Courier', font_size=20, x=150, y=150)
        Utils.create_label(window=self.canvas, default_text='X', font='Courier', font_size=20, x=215, y=150)
        
        self.x_textbox = Utils.create_textbox(window=self.canvas, height=1, width=2, default_text='1', 
                                             font='Courier', font_size=20, x=110, y=150)
        self.y_textbox = Utils.create_textbox(window=self.canvas, height=1, width=2, default_text='1', 
                                             font='Courier', font_size=20, x=175, y=150)
        self.z_textbox = Utils.create_textbox(window=self.canvas, height=1, width=2, default_text='1', 
                                             font='Courier', font_size=20, x=240, y=150)
        
        create_btn = Button(self.canvas, text='Create', width=19, height=1, bg='cyan',
                           command=self.create_furniture)
        create_btn.configure(font=('Courier', 20))
        create_btn.place(x=20, y=0)
        
    def create_furniture(self):
        size_w = Utils.get_textbox(self.x_textbox, to_int=True)
        size_h = Utils.get_textbox(self.y_textbox, to_int=True)
        
        grid_module = self.parent.grid_module
        frame_position = grid_module.frame_position
        frame_size = grid_module.frame_size
        grid_size = grid_module.get_gridsize()
        
        if size_w <= 0:
            size_w = 1
            Utils.set_textbox(self.x_textbox, txt=str(1))
        elif size_w > grid_module.grid_count['x']:
            size_w = grid_module.grid_count['x']
            Utils.set_textbox(self.x_textbox, txt=str(size_w))
        
        if size_h <= 0:
            size_h = 1
            Utils.set_textbox(self.y_textbox, txt=str(1))
        elif size_h > grid_module.grid_count['y']:
            size_h = grid_module.grid_count['y']
            Utils.set_textbox(self.y_textbox, txt=str(size_h))            
        
        xlist = [x+frame_position['x']+5 for x in range(0, frame_size['w'], int(grid_size['w']))]
        ylist = [y+frame_position['y']+5 for y in range(0, frame_size['h'], int(grid_size['h']))]
        
        obj_size = {'w':size_w*grid_size['w']-1, 'h':size_h*grid_size['h']-1}
        init_pos = {'x':frame_position['x']-grid_size['w'],'y':frame_position['y']-grid_size['h']}
        can_pos = {'xlist':xlist, 'ylist':ylist}
        
        furniture = Furniture(window=self.window, parent = self.parent, bg='blue', 
                              size=obj_size, init_pos=init_pos, can_pos=can_pos)
        
        self.parent.furniture_list.append(furniture)
        self.parent.furniture_objlist.append(furniture.get_object())
        self.parent.focus = furniture

In [172]:
class ListModule():
    def __init__(self, window, parent, list_x=0, list_y=0, list_width=100, list_height=100):
        self.window = window
        self.parent = parent
        self.canvas = Utils.create_canvas(window=window, width=list_width, height=list_height,
                                         x=list_x, y=list_y)
        
        self.canvas_position = {'x':list_x, 'y':list_y}
        self.canvas_size = {'w':list_width, 'h':list_height}
        
        self.frame = Frame(window)
        self.frame.grid(row=0, column=0, sticky='n')
        
        self.furniture_list = pd.read_csv('color.csv')
        
        rows=8
        columns=2
        n=0
        
        for i in range(0, rows):
            for j in range(0, columns):
                self.make_button(canvas=self.canvas, n=n, row=i, column=j)
                n+=1
                
        self.frame.grid_columnconfigure(1, minsize=20)
        
    def get_canvas_size(self):
        minx = self.canvas_position['x']
        miny = self.canvas_position['y']
        maxx = self.canvas_position['x'] + self.canvas_size['w']
        maxy = self.canvas_position['y'] + self.canvas_size['h']
        return {'min':{'x':minx, 'y':miny}, 'max':{'x':maxx, 'y':maxy}}
        
    def make_button(self, canvas, n, row, column):
        text = self.furniture_list['Type'][n]
        bg = self.furniture_list['Color'][n]
        active = self.furniture_list['ActiveColor'][n]
        font_color = self.furniture_list['TextColor'][n]
        
        btn = Button(canvas, text=text, command=lambda:self.click(btn), width=15, height=4)
        btn.config(font=('Courier', 14), bg=bg, activebackground=active, fg=font_color, activeforeground=font_color)
        btn.grid(row=row, column=column, sticky='we')
        
    def click(self, button):
        if self.parent.focus:
            focus = self.parent.focus
            focus_object = focus.get_object()

            text = button.cget('text')
            color = button.cget('bg')
            font_color = button.cget('fg')
            focus.org_color = color
            focus.org_text_color = font_color

            focus_object.config(bg=color)
            focus_object.itemconfig(focus_object.find_withtag(focus.text), text=text, 
                                    fill=font_color, font=('Courier', 10))
            
            self.parent.focus = None
        

In [115]:
class ModifyModule():
    def __init__(self, window, parent):
        self.window = window
        self.parent = parent        
        self.canvas = None
        
    def right_click(self):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        
        if self.canvas == None:
            self.canvas = Utils.create_canvas(window=self.window, width=100, height=50, bg='cyan', x=mouse_x, y=mouse_y)
            self.make_module()
            
        else:
            self.delete_module()
            self.canvas = None        
        
    def make_module(self):
        self.delete_button = Button(self.canvas, text='Delete')
        self.delete_button.place(x=0, y=0)        
        self.delete_button.config(bg='green', font=('Courier', 14))
        
    def delete(self):
        self.delete_module()
        self.canvas.destroy()
        self.parent.obj.destroy()
        
        #furniture list, obj lsit, focus object 해제
    
    def delete_module(self):
        self.delete_button.destroy()
        self.canvas.destroy()

In [177]:
class Widget():
    def __init__(self, window, parent, widget_x=0, widget_y=0, widget_width=100, widget_height=100):
        self.window = window
        self.parent = parent
        self.canvas = Utils.create_canvas(window=window, width=widget_width, height=widget_height, x=widget_x, y=widget_y)
        
        Utils.create_label(window=self.canvas, default_text='RoomSize', font='Courier', font_size=20, x=250, y=0)
        Utils.create_label(window=self.canvas, default_text='X', font='Courier', font_size=20, x=320, y=50)
        
        self.grid_x_textbox = Utils.create_textbox(window=self.canvas, width=7, height=1, default_text='10', 
                                                  font='Courier', font_size=20, x=200, y=50)
        self.grid_y_textbox = Utils.create_textbox(window=self.canvas, width=7, height=1, default_text='10',
                                                  font='Courier', font_size=20, x=345, y=50)
        submit_btn = Button(self.canvas, text='Submit', font=("Courier", 14), command=self.submit)
        submit_btn.place(x=470, y=50)
        
        next_btn = Button(self.window, text='Next', font=("Courier", 14), command=self.parent.next_btn)
        next_btn.place(x=1000, y=850)
        
    def submit(self):
        self.parent.clear_object()
        grid_module = self.parent.grid_module
        
        cnt_grid_x = Utils.get_textbox(self.grid_x_textbox, to_int=True)
        cnt_grid_y = Utils.get_textbox(self.grid_y_textbox, to_int=True)
        
        grid_limit = grid_module.grid_limit
        
        if cnt_grid_x > grid_limit['max']:
            cnt_grid_x = grid_limit['max']
            Utils.set_textbox(self.grid_x_textbox, str(grid_limit['max']))
        elif cnt_grid_x < grid_limit['min']:
            cnt_grid_x = grid_limit['min']
            Utils.set_textbox(self.grid_x_textbox, str(grid_limit['min']))
            
        if cnt_grid_y > grid_limit['max']:
            cnt_grid_y = grid_limit['max']
            Utils.set_textbox(self.grid_y_textbox, str(grid_limit['max']))
        elif cnt_grid_y < grid_limit['min']:
            cnt_grid_y = grid_limit['min']
            Utils.set_textbox(self.grid_y_textbox, str(grid_limit['min']))
        
        grid_module.grid_count = {'x':cnt_grid_x, 'y':cnt_grid_y}
        grid_module.update_gridsize()
        self.parent.grid_module.make_grid()

In [178]:
class Furniture():
    def __init__(self, window, parent, init_pos, can_pos, bg='blue', size={'w':0, 'h':0}):
        self.window = window
        self.parent = parent
        
        self.position = {'x':init_pos['x'], 'y':init_pos['y']}
        self.size = {'w':size['w'], 'h':size['h']}
        self.can_pos = can_pos
        
        self.obj= Utils.create_canvas(window=window, width=self.size['w'], height=self.size['h'],
                                     x=self.position['x'], y=self.position['y'], bg=bg)
        self.modify_module = ModifyModule(window=self.window, parent=self)
        
        self.obj.bind('<Button-1>', self.click)
        self.obj.bind('<Button-3>', self.modify)
        self.obj.bind('<B1-Motion>', self.move)
        self.obj.bind('<ButtonRelease-1>', self.release)
        
        self.org_color = 'blue'
        self.org_text_color='black'
        self.text_pos = {'x':self.size['w']/2, 'y':self.size['h']/2}
        self.text = self.obj.create_text(self.text_pos['x'], self.text_pos['y'],
                                        fill='black', text='')
        
    def get_config(self):
        data  = [
            {'position':self.position},
            {'size': self.size},
            {'type':self.obj.itemcget(self.text, 'text')}
        ]
        
        return data
        
    def get_object(self):
        return self.obj
    
    def modify(self, event):
        self.modify_module.right_click()
        
    def update_position(self):
        self.obj.place(x=self.position['x'], y=self.position['y'])
    
    def click(self, event):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        
        self.diff_x = int(mouse_x) - self.position['x']
        self.diff_y = int(mouse_y) - self.position['y']
        
        for furniture in self.parent.furniture_list:
            furniture.get_object().config(bg=furniture.org_color)
        self.parent.focus = self
        self.obj.itemconfig(self.obj.find_withtag(self.text), fill='white')
        self.obj.config(bg='purple')
    
    def move(self, event):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        self.position['x'] = int(mouse_x) - self.diff_x
        self.position['y'] = int(mouse_y) - self.diff_y
        self.update_position()
        
    def release(self, event):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        
        grid_canvas_size = self.parent.grid_module.get_canvas_size()
        
        if mouse_x < grid_canvas_size['min']['x'] - self.size['w']/2 or mouse_x > grid_canvas_size['max']['x']+self.size['w']/2 or mouse_y < grid_canvas_size['min']['y'] - self.size['h']/2 or mouse_y > grid_canvas_size['max']['y'] + self.size['h']/2:
            return
        
        else:            
            if mouse_x < grid_canvas_size['min']['x'] and mouse_y < grid_canvas_size['min']['y']:
                self.position['x'] = grid_canvas_size['min']['x'] + 6
                self.position['y'] = grid_canvas_size['min']['y'] + 6
            elif mouse_x > grid_canvas_size['max']['x'] and mouse_y > grid_canvas_size['max']['y']:
                self.position['x'] = grid_canvas_size['max']['x'] - self.size['w'] + 5
                self.position['y'] = grid_canvas_size['max']['y'] - self.size['h'] + 5
            elif mouse_x < grid_canvas_size['min']['x']:
                self.position['x'] = grid_canvas_size['min']['x'] + 6
            elif mouse_x > grid_canvas_size['max']['x']:
                self.position['x'] = grid_canvas_size['max']['x'] - self.size['w'] + 5                
            elif mouse_y < grid_canvas_size['min']['y']:
                self.position['y'] = grid_canvas_size['min']['y'] + 6
            elif mouse_y > grid_canvas_size['max']['y']:
                self.position['y'] = grid_canvas_size['max']['y'] - self.size['h'] + 5
            
            if mouse_x >= grid_canvas_size['min']['x'] and mouse_x <= grid_canvas_size['max']['x']:
                place_x = self.can_pos['xlist'][0]
                for x in self.can_pos['xlist']:
                    if self.position['x'] > x:
                        place_x = x
                self.position['x'] = place_x + 1
                
            if mouse_y >= grid_canvas_size['min']['y'] and mouse_y <= grid_canvas_size['max']['y']:
                place_y = self.can_pos['ylist'][0]
                for y in self.can_pos['ylist']:
                    if self.position['y'] > y:
                        place_y = y
                self.position['y'] = place_y + 1
                
        self.update_position()

In [179]:
class Application():
    def __init__(self, title='Furniture Layout Batch Application',
                window_width=100, window_height=100, window_x=0, window_y=0, is_resize=False):
        geometry = Utils.size_to_geometry(window_width=window_width, window_height=window_height, 
                                         window_x=window_x, window_y=window_y)
        self.window = Utils.create_window(title=title, geometry=geometry, is_resize=is_resize)
        
        self.grid_module = GridModule(window=self.window, parent=self, grid_x=450, grid_y=200, grid_width=600, grid_height=600)
        self.widget = Widget(window=self.window, parent=self, widget_x=450, widget_y=50, widget_width=600, widget_height=100)
        self.list_module = ListModule(window=self.window, parent = self, list_x=50, list_y=80, list_width=350, list_height=800)
        self.create_module = CreateModule(window=self.window, parent=self, create_x=1100, create_y=80,
                                         create_width=350, create_height=800)
        
        self.furniture_list = []
        self.furniture_objlist = []
        self.focus = None
        
        self.window.bind('<Button-1>', self.click)
        
        self.grid = [[0 for x in range(10)] for y in range(10)]

    def runApp(self):
        self.window.mainloop()
        
    def click(self, event):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        
        grid_canvas_size = self.grid_module.get_canvas_size()
        list_canvas_size = self.list_module.get_canvas_size()
        
        if mouse_x <= grid_canvas_size['min']['x'] or mouse_x >= grid_canvas_size['max']['x'] or mouse_y <= grid_canvas_size['min']['y'] or mouse_y >= grid_canvas_size['max']['y']:
            if mouse_x <= list_canvas_size['min']['x'] or mouse_x >= list_canvas_size['max']['x'] or mouse_y <= list_canvas_size['min']['y'] or mouse_y >= list_canvas_size['max']['y']:
                if self.focus:
                    self.focus.get_object().config(bg=self.focus.org_color)
                    self.focus.get_object().itemconfig(self.focus.get_object().find_withtag(self.focus.text), fill=self.focus.org_text_color)
                self.focus = None
                
    def clear_object(self):
        for furniture in self.furniture_list:
            furniture.get_object().destroy()
        self.furniture_list = []
        self.furniture_objlist = []
        self.focus = None
        
    def next_btn(self):        
        grid_config = self.grid_module.get_config()
        
        furniture_configs = []
        for furniture in self.furniture_list:
            furniture_configs.append(furniture.get_config())
        
        self.clear_object()
        self.window.destroy()

In [180]:
app = Application(window_width=1500, window_height=1000, window_x=200, window_y=200)
app.runApp()

[{'position': {'x': 450, 'y': 200}}, {'size': {'w': 600, 'h': 600}}, {'grid_count': {'x': 10, 'y': 10}}, {'grid_size': {'w': 60.0, 'h': 60.0}}]


In [105]:
if x<minx-size['w']/2 or x>maxx+size['w']/2 or y<miny-size['h']/2 or y>maxy+size['h']/2:
    not do
else:
    if x<minx and y<miny => x,y 둘다 가져오고 세팅
    elif x>maxx and y>maxy => x,y 둘다 가져오고 세팅
    elif x<minx: x만
    elif x>maxx: x만
    elif y<miny: y만
    elif y>miny: y만

    if minx<x<maxx: x위치
    if miny<y<maxy: y위치

    update

포지션이 maxpos + 현재 object 크기 라면? 해당 위치에 삽입하고


SyntaxError: invalid syntax (Temp/ipykernel_8568/3020724025.py, line 4)